In [1]:
# coding: utf-8
"""
考察部分で使う為のデータを取得する為のコード
全ての分の割り当てを行う
"""
from igraph import *
import csv
import collections
import pickle
import numpy as np
from openopt import QP
import copy
import re

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def readdump(path):
    f = open(path, "r")
    arr = pickle.load(f)
    f.close()
    return arr

# 有向エッジリストを入力して、重み付き無向ネットワークを出力する
def cal_edgelist_to_network(list_edge):
    # 有向エッジリストを無向エッジリストに変換する
    list_edge = [tuple(sorted(row)) for row in list_edge]
    # ノードリスト
    list_vertices = list(set([word for row in list_edge for word in row]))
    # エッジリストとそのweightを作成
    tuple_edge, tuple_weight = zip(*collections.Counter(list_edge).items())
    return {"vertex": list_vertices, "edge": list(tuple_edge), "weight": list(tuple_weight)}
    
    
# クラスタリング済みのネットワークを元にサブグラフのリストを作成
# vertexには全てのvertexを代入する（PageRankを計算するため）
def cal_cluster_to_network(dict_network):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # クラスタごとにwordをまとめる
    dict_cluster = collections.defaultdict(list)
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_cluster[cluster].append(word)

    # リストに変換
    list_cluster_vertex = [row[1] for row in dict_cluster.items()]
    
    # 同様にエッジとウェイトのリストも作成する
    list_cluster_edge = []
    list_cluster_weight = []
    for cluster_vertex in list_cluster_vertex:
        list_cluster_edge_one = []
        list_cluster_weight_one = []
        # エッジリストの中に、一つでもノードが含まれていれば、そのクラスのノードに含める
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            # and と or を切り替えることによって性能の比較
            if row[0] in cluster_vertex or row[1] in cluster_vertex:
                list_cluster_edge_one.append(row)
                list_cluster_weight_one.append(weight)
        list_cluster_edge.append(list_cluster_edge_one)
        list_cluster_weight.append(list_cluster_weight_one)
    
    # まとめる
    list_dict_network = [{"vertex": dict_network["vertex"],
                          "edge": cluster_edge,
                          "weight": cluster_weight}
                         for cluster_edge, cluster_weight
                         in zip(list_cluster_edge, list_cluster_weight)]
    
    return list_dict_network

# f_measureを計算する
def cal_f_measure(list_predict_measure):
    # 生成したクラスタ内のカウント
    dict_predict_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_predict_cluster[row[0]].append(row[1])
        
    for key, row in dict_predict_cluster.items():
        print key, sorted(collections.Counter(row).items(), key=lambda x: x[0])
    print "================="
    
    # もとあるクラス内のカウント
    dict_measure_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_measure_cluster[row[1]].append(row[0])
    
    for key, row in dict_measure_cluster.items():
        print key, sorted(collections.Counter(row).items(), key=lambda x: x[0])
    
    # local_purityの計算
    list_purity = []
    for row in dict_predict_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_purity.append([major_class, class_num])
    purity = float(np.sum(zip(*list_purity)[0])) / np.sum(zip(*list_purity)[1])
    print "Purity: ", purity
    
    # inverse_purityの計算
    list_inverse_purity = []
    for row in dict_measure_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_inverse_purity.append([major_class, class_num])
    inverse_purity = float(np.sum(zip(*list_inverse_purity)[0])) / np.sum(zip(*list_inverse_purity)[1])
    print "Inverse Purity: ", inverse_purity
    
    print "F-value: ", 2 / (1 / purity + 1 / inverse_purity)
    return purity, inverse_purity, 2 / (1 / purity + 1 / inverse_purity)
    
# 凸２次計画問題を解いてp(topic)を求めるための関数
def cal_prob_topic(dict_network_master, list_dict_network_sub):
    prob_master = np.array([row[1] for row in sorted(zip(dict_network_master["vertex"], dict_network_master["page_rank"]), key=lambda x: x[0])])
    
    for i, dict_network_sub in enumerate(list_dict_network_sub):
        if i == 0:
            prob_sub = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
        else:
            list_tmp = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
            prob_sub = np.vstack((prob_sub, list_tmp))
    
    H = 2 * prob_sub.dot(prob_sub.T)
    f = -2 * prob_master.dot(prob_sub.T)
    Aeq = np.ones(len(list_dict_network_sub))
    beq = 1
    lb = np.zeros(len(list_dict_network_sub))
    
    p = QP(H, f, Aeq=Aeq, beq=beq, lb=lb)
    r = p.solve("cvxopt_qp")
    k_opt = r.xf
    return k_opt

In [2]:
# クラスタリング後に、ノードごとにどのクラスタにエッジを持っているのか計算
def cal_cluster_to_edge(dict_network, low_fleq=0.05, low_rate=0.8, flag=0):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # wordをclusterに変換するための辞書を作成する
    dict_word_to_cluster = {}
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_word_to_cluster[word] = cluster
        
    # wordをidに変換するための辞書を作成する
    dict_word_to_id = {}
    for i, word in enumerate(dict_network["vertex"]):
        dict_word_to_id[word] = i
    
    # clusterの数
    num_cluster = len(set(dict_network["cluster"]))
    
    # id_to_clusterのマトリックス
    matrix_id_to_cluster = np.zeros((len(dict_word_to_id), num_cluster))
    for row, weight in zip(dict_network["edge"], dict_network["weight"]):
        matrix_id_to_cluster[dict_word_to_id[row[0]]][dict_word_to_cluster[row[1]]] += weight
        matrix_id_to_cluster[dict_word_to_id[row[1]]][dict_word_to_cluster[row[0]]] += weight
        
    #総単語数を求める
    total_voc = np.sum(matrix_id_to_cluster)
    
    # 指定したハイパーパラメータよりも高い値を記録した単語を所属クラスタとともに辞書に登録する
    dict_word_to_list_cluster = {}
    for cluster, word, row in zip(dict_network["cluster"], dict_network["vertex"], matrix_id_to_cluster):
        top_num = max(row)
        if float(np.sum(row))/total_voc>=low_fleq and len(np.where(row/top_num>=low_rate)[0])>=2:
            #print word
            dict_word_to_list_cluster[word] = np.where(row/top_num>=low_rate)[0]
            
    # 分割する単語が存在するかしないかflagを立てる
    if len(dict_word_to_list_cluster) > 0:
        outflag = True
    else:
        outflag = False

    # 新しく分裂するノードクラスターを元のクラスターに記録
    for word, row in dict_word_to_list_cluster.items():
        for num in row:
            # dict_word_to_cluster[word+"_"+str(num)] = num
            dict_word_to_cluster[word+"_"+str(num)] = dict_word_to_cluster[word]
    
    # ここの計算が間違ってそう => diffsplitで確認したが大丈夫そう・・・
    for word in dict_word_to_list_cluster.keys():
        list_edge_new = []
        list_weight_new = []
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            if row[0] == word:
                if dict_word_to_cluster[row[1]] in dict_word_to_list_cluster[word]:
                    list_edge_new.append([row[0]+"_"+str(dict_word_to_cluster[row[1]]), row[1]])
                    list_weight_new.append(weight)
                else:
                    for num in dict_word_to_list_cluster[word]:
                        # flagの値によって、weightの計算法を切り替え
                        if flag == 0:
                            weight_tmp = float(weight)/len(dict_word_to_list_cluster[word])
                            list_edge_new.append([row[0]+"_"+str(num), row[1]])
                            list_weight_new.append(weight_tmp)
                        else:
                            weight_tmp = weight/len(dict_word_to_list_cluster[word])
                            if weight_tmp > 0:
                                list_edge_new.append([row[0]+"_"+str(num), row[1]])
                                list_weight_new.append(weight_tmp)
                                
            elif row[1] == word:
                if dict_word_to_cluster[row[0]] in dict_word_to_list_cluster[word]:
                    list_edge_new.append([row[0], row[1]+"_"+str(dict_word_to_cluster[row[0]])])
                    list_weight_new.append(weight)
                else:
                    for num in dict_word_to_list_cluster[word]:
                        # flagの値によって、weightの計算法を切り替え
                        if flag == 0:
                            weight_tmp = float(weight)/len(dict_word_to_list_cluster[word])
                            list_edge_new.append([row[0], row[1]+"_"+str(num)])
                            list_weight_new.append(weight_tmp)
                        else:
                            weight_tmp = weight/len(dict_word_to_list_cluster[word])
                            if weight_tmp > 0:
                                list_edge_new.append([row[0], row[1]+"_"+str(num)])
                                list_weight_new.append(weight_tmp)
            else:
                list_edge_new.append([row[0], row[1]])
                list_weight_new.append(weight)
        else:
            dict_network["edge"] = copy.deepcopy(list_edge_new)
            dict_network["weight"] = copy.deepcopy(list_weight_new)
            list_vertices = list(set([word for row in dict_network["edge"] for word in row]))
            dict_network["vertex"] = list_vertices
    
    return dict_network, outflag

###メイン部分
1. 有向エッジリストから無向重み付きエッジリストを作成
2. louvain法によるクラスタリング
3. クラスタごとにpagerankを計算し、p(word|topic)とする
4. クラスタごとにp(word|topic)を出力する辞書を作成

In [11]:
def main(path, testpath, alpha, beta, n=0, p_k=True, DN=True): 
    # エッジリストの読み込み
    list_edge = readcsv(path)
    # 元のネットワークを作成する（無向）
    dict_network_master = cal_edgelist_to_network(list_edge)
    # g = Graph(directed=True)
    g_master = Graph()
    g_master.add_vertices(dict_network_master["vertex"])
    g_master.add_edges(dict_network_master["edge"])
    # 元のネットワークのpagerankを求める
    dict_network_master["page_rank"] = g_master.pagerank(directed=False, weights=dict_network_master["weight"])

    counter=0
    while DN:
        # louvain法によるクラスタリング、vertexと同じ長さのクラスタ番号が書かれたリストがreturn
        if n == 0:
            dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering().membership
        else:
            dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering(n=n).membership
        # クラスタリングの結果をもとに、どのノードから何本のエッジが出ているか計算する
        dict_network_master, DN = cal_cluster_to_edge(dict_network_master, low_fleq=alpha, low_rate=beta, flag=0)
        counter +=1
        #print counter, "回目"
        #print len(dict_network_master["weight"])
        #print len(dict_network_master["edge"])
        g_master = Graph()
        g_master.add_vertices(dict_network_master["vertex"])
        g_master.add_edges(dict_network_master["edge"])
    if n == 0:
        dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering().membership
    else:
        dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering(n=n).membership
    # 元のネットワークのpagerankを求める
    dict_network_master["page_rank"] = g_master.pagerank(directed=False, weights=dict_network_master["weight"])
    # クラスタ結果をもとにサブグラフのリストを作成
    list_dict_network_sub = cal_cluster_to_network(dict_network_master)
    # サブクラスタごとに中心性の計算
    for i, dict_network_sub in enumerate(list_dict_network_sub):
        g_sub = Graph()
        g_sub.add_vertices(dict_network_sub["vertex"])
        g_sub.add_edges(dict_network_sub["edge"])
        list_dict_network_sub[i]["page_rank"] = g_sub.pagerank(directed=False, weights=dict_network_sub["weight"])
    print "クラスタ数: ", len(list_dict_network_sub)

    # トピックごとにwordを入力したらp(word|topic)が出るような辞書を作成
    list_dict_prob = []
    for i in range(len(list_dict_network_sub)):
        list_word_page = sorted(zip(list_dict_network_sub[i]["vertex"], list_dict_network_sub[i]["page_rank"]), key=lambda x: x[1], reverse=True)
        list_word_page_rev = []
        for row in list_word_page:
            pattern = "_[0-9|_]+"
            word = re.sub(pattern, "", row[0])
            list_word_page_rev.append([word, row[1]]) 
        list_dict_prob.append({row[0]: row[1] for row in list_word_page_rev})
    
    # 凸２次計画問題を解いて、p(topic)を求める
    list_prob_topic = cal_prob_topic(dict_network_master, list_dict_network_sub)
    print "事前分布: ", list_prob_topic
    list_dict_words = dict_network_master["vertex"]
    #list_sep_words = readdump(testpath)
    list_sep_words = readcsv("./files/rakuten_corpus/rakuten_corpus_for_UM.txt")
    list_sep_words_rev = []
    for row in list_sep_words:
        list_sep_words_rev.append(row[0].split(" "))
    # 確率が最大になるクラスを予想する
    # 実質ラベルがintじゃない場合は、エラーとして、記録しない
    list_predict = []
    list_words_rev = []
    error_count = 0
    for row in list_sep_words_rev:
        class_tmp = 0
        prob_tmp = 0
        predict_class = []
        for num, dict_prob in enumerate(list_dict_prob):
            prob_tmp_tmp = 1
            for word in row:
                try:
                    prob_tmp_tmp *= float(dict_prob[word])
                except KeyError:
                    pass
            if p_k:
                prob_tmp_tmp *= list_prob_topic[num]
            if prob_tmp_tmp > prob_tmp:
                class_tmp = num
                prob_tmp = prob_tmp_tmp
        list_predict.append(class_tmp)
    #print "エラーデータ数: ", error_count
    #print "テストデータ数: ",len(list_predict_measure)
    return list_predict

### クラスタ数指定なしで最適だと判断されたF-value

In [14]:
list_predict = main("./files/rakuten_corpus/rakuten_corpus_edgelist.csv", 
                           "./files/rakuten_corpus/annotation/all_sep_others.dump",
                           alpha=0.002, beta=0.6, n=0, p_k=True, DN=True)

print sorted(collections.Counter(list_predict).items(), key=lambda x:x[0])

クラスタ数:  10

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -2.1674e-03 -1.0038e+00  1e+00  8e-17  3e+00
 1: -2.1694e-03 -1.3776e-02  1e-02  5e-17  4e-02
 2: -2.2949e-03 -3.5358e-03  1e-03  7e-17  4e-03
 3: -2.5673e-03 -2.7433e-03  2e-04  5e-17  1e-18
 4: -2.6124e-03 -2.6346e-03  2e-05  9e-17  1e-18
 5: -2.6220e-03 -2.6251e-03  3e-06  8e-17  2e-18
 6: -2.6237e-03 -2.6240e-03  4e-07  7e-17  1e-18
 7: -2.6239e-03 -2.6240e-03  2e-08  4e-17  1e-18
 8: -2.6240e-03 -2.6240e-03  4e-10  1e-16  1e-18
Optimal solution found.
istop: 1000 (optimal)
Solver:   Time Elapsed = 0.0 	CPU Time Elapsed = 0.0
objFuncValue: -0.0026239608 (feasible, MaxResidual = 1.11022e-16)
事前分布:  [  9.14823265e-02   1.92637203e-01   1.85392253e-01   2.41471130e-01
   2.03989574e-01   8.50185603e-02   1.78136279e-06   2.87888128e-07
   6.38049840e-06   5.03553814e-07]
[(0, 308), (1, 706), (2, 847), (3, 1322)

In [13]:
list_predict = main("./files/rakuten_corpus/rakuten_corpus_edgelist.csv", 
                           "./files/rakuten_corpus/annotation/all_sep_others.dump",
                           alpha=0.002, beta=0.6, n=12, p_k=True, DN=True)

print sorted(collections.Counter(list_predict).items(), key=lambda x:x[0])

クラスタ数:  12

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -2.0446e-03 -1.0040e+00  1e+00  4e-17  4e+00
 1: -2.0468e-03 -1.4016e-02  1e-02  7e-17  4e-02
 2: -2.1909e-03 -3.7447e-03  2e-03  5e-17  6e-03
 3: -2.5406e-03 -2.8065e-03  3e-04  7e-17  2e-18
 4: -2.6052e-03 -2.6427e-03  4e-05  6e-17  1e-18
 5: -2.6208e-03 -2.6259e-03  5e-06  7e-17  6e-19
 6: -2.6235e-03 -2.6241e-03  6e-07  7e-17  9e-19
 7: -2.6239e-03 -2.6240e-03  4e-08  1e-16  1e-18
 8: -2.6240e-03 -2.6240e-03  7e-10  7e-17  1e-18
Optimal solution found.
istop: 1000 (optimal)
Solver:   Time Elapsed = 0.0 	CPU Time Elapsed = 0.0
objFuncValue: -0.002623962 (feasible, MaxResidual = 0)
事前分布:  [  9.18422560e-02   1.92164066e-01   1.85512207e-01   2.41128813e-01
   2.04060671e-01   8.52741278e-02   3.39145541e-06   5.75165469e-07
   1.22931049e-05   3.14901613e-07   2.73696181e-07   1.01095450e-06]
[(0, 308), (1, 70

### bigramではなく、共起グラフを用いた場合

In [14]:
list_purity = []
list_inv = []
list_fvalue = []
for n in range(3, 21):
    purity, inv, fvalue = main("./files/rakuten_corpus/rakuten_corpus_edgelist.csv", 
                               "./files/rakuten_corpus/annotation/all_sep.dump",
                               alpha=0.002, beta=0.6, n=n, p_k=True, DN=True)
    list_purity.append([purity])
    list_inv.append([inv])
    list_fvalue.append([fvalue])
writecsv(list_purity, "./files/purity_DNP+p(k).csv")
writecsv(list_inv, "./files/inv_DNP+p(k).csv")
writecsv(list_fvalue, "./files/fvalue_DNP+p(k).csv")

クラスタ数:  3

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -2.5746e-03 -1.0031e+00  1e+00  1e-16  2e+00
 1: -2.5752e-03 -1.3054e-02  1e-02  1e-16  2e-02
 2: -2.6092e-03 -3.0534e-03  4e-04  2e-16  1e-03
 3: -2.6489e-03 -2.6749e-03  3e-05  2e-16  3e-06
 4: -2.6494e-03 -2.6497e-03  3e-07  8e-17  3e-08
 5: -2.6494e-03 -2.6494e-03  3e-09  8e-17  3e-10
Optimal solution found.
istop: 1000 (optimal)
Solver:   Time Elapsed = 0.0 	CPU Time Elapsed = 0.0
objFuncValue: -0.0026493995 (feasible, MaxResidual = 0)
テストデータ数:  1998
Purity:  0.447447447447
Inverse Purity:  0.72022022022
F-value:  0.551973319226
クラスタ数:  4

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -2.3583e-03 -1.0036e+00  1e+00  4e-16  3e+00
 1: -2.3596e-03 -1.3600e-02  1e-02  8e-17  3e-02
 2: -2.4337e-0

### クラスタ数を指定して、変化させた場合の結果

In [19]:
for n in range(3, 21):
    purity, inv, fvalue = main("./files/rakuten_corpus/rakuten_corpus_edgelist_window1.csv", 
             "./files/rakuten_corpus/annotation/all_sep.dump",
             alpha=0.002, beta=0.6, n=n)

クラスタ数:  3

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -1.9617e-03 -1.0027e+00  1e+00  6e-17  2e+00
 1: -1.9623e-03 -1.2648e-02  1e-02  2e-16  2e-02
 2: -1.9980e-03 -2.5926e-03  6e-04  2e-16  1e-03
 3: -2.0467e-03 -2.0818e-03  4e-05  6e-17  9e-19
 4: -2.0471e-03 -2.0475e-03  4e-07  8e-17  4e-19
 5: -2.0471e-03 -2.0471e-03  4e-09  2e-16  3e-19
Optimal solution found.
istop: 1000 (optimal)
Solver:   Time Elapsed = 0.0 	CPU Time Elapsed = 0.0
objFuncValue: -0.0020471111 (feasible, MaxResidual = 0)
テストデータ数:  1998
Purity:  0.455455455455
Inverse Purity:  0.745245245245
F-value:  0.56537988593
クラスタ数:  4

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -1.9006e-03 -1.0024e+00  1e+00  6e-17  3e+00
 1: -1.9008e-03 -1.2416e-02  1e-02  1e-16  3e-02
 2: -1.9185e-0